In [6]:
import polars as pl
from typing import List, Tuple, Union, Optional, Callable, Any, Dict
from datetime import datetime
import uuid
from llama_cpp import Llama
import polars_distance as pld


In [2]:
llm = Llama(model_path="/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf",chat_format="chatml",n_ctx=0,n_gpu_layers = -1, use_mlock=True)

llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from /Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ehartford_dolphin-2.5-mixtral-8x7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dim

In [3]:
#the schema of the df is: 
messages = [
          {"role": "system", "content": "You are an assistant who perfectly describes python packages. GANG GANG"},
          {
              "role": "user",
              "content": "What is polars library in python?"
          }
      ]
results = llm.create_chat_completion(
      messages = messages,
)



llama_print_timings:        load time =    2430.92 ms
llama_print_timings:      sample time =      25.54 ms /   252 runs   (    0.10 ms per token,  9866.10 tokens per second)
llama_print_timings: prompt eval time =    2430.71 ms /    37 tokens (   65.69 ms per token,    15.22 tokens per second)
llama_print_timings:        eval time =   11095.01 ms /   251 runs   (   44.20 ms per token,    22.62 tokens per second)
llama_print_timings:       total time =   13958.61 ms


In [4]:
dir(llm.n_ctx())

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_count',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'is_integer',
 

In [5]:
results

{'id': 'chatcmpl-4107df95-31b0-4c21-80ff-d59184847316',
 'object': 'chat.completion',
 'created': 1704801133,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': ' Polars is a fast, efficient, and easy-to-use data processing library for Python. It provides a DataFrame object similar to pandas but optimized for speed and memory efficiency. Polars uses Rust as its core language, which allows it to perform computations much faster than other Python libraries. The library supports various operations like filtering, groupby, joins, and more on large datasets with ease.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 37, 'completion_tokens': 86, 'total_tokens': 123}}

In [5]:
def vector_search(df: pl.DataFrame, embedding_column:str, query_embedding: List[float]):
    target_lit = pl.lit(pl.Series("query", [query_embedding]))
    df_with_query = df.with_columns(target_lit.alias("query")).cast({"query": pl.Array(inner=pl.Float64, width=3)})
    df_distance = df_with_query.with_columns(pld.col(embedding_column).dist_arr.euclidean('query').alias('dist'))
    return df_distance.sort('dist')


class ChatFrame:
      chat_schema = {"role":pl.Utf8,
                   "content":pl.Utf8,
                   "time_stamps":pl.Datetime,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   "prompt": pl.List(pl.Struct({"role":pl.Utf8,"content":pl.Utf8})),
                   'prompt_tokens': pl.Int64, 
                   'completion_tokens': pl.Int64, 
                   'total_tokens': pl.Int64
}     
      chat_ml_schema =  {"chat_ml":pl.Struct({"role":pl.Utf8,"content":pl.Utf8}),
                   "time_stamps":pl.Datetime,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   'completion_tokens': pl.Int64, 
}
      chat_nested_schema = {"role":pl.Utf8,
                   "content":pl.Utf8,
                   "time_stamps":pl.Datetime,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   "prompt": pl.List(pl.Struct({"role":pl.Utf8,"content":pl.Utf8})),
                  "usage": pl.Struct({'prompt_tokens': pl.Int64, 'completion_tokens': pl.Int64, 'total_tokens': pl.Int64})
}
      def __init__(self,llm: Llama):
            self.chat_id = str(uuid.uuid4())
            # self.messages = chat_ml_messages
            # self.time_stamps = self.validate_timestamps(time_stamps)
            self.df = pl.DataFrame(schema=self.chat_schema)
            self.llm = llm
            self.ctx = llm.n_ctx()
            self.output_reserved = 1500
                        
      
      def add_from_chat_ml(self,chat_ml_messages: List[Dict[str, str]] , 
                           time_stamps: Optional[List[datetime]]=None,
                           author: Optional[List[str]]=None,):
            if time_stamps is None:
                  time_stamps = [datetime.now() for _ in range(len(chat_ml_messages))]

            message_id = ["chatml-" + str(uuid.uuid4()) for _ in range(len(chat_ml_messages))]
            if author is None:
                  author = ["import" for _ in range(len(chat_ml_messages))]
            chat_id = [self.chat_id for _ in range(len(chat_ml_messages))]
            
            message_tokens_len = [len(self.llm.tokenize(bytes(message["role"]+ message["content"], "utf-8"))) for message in chat_ml_messages]
            

            nested_df = pl.DataFrame({"chat_ml":chat_ml_messages,
                                      "time_stamps":time_stamps,
                                      "author":author,
                                      "message_id":message_id,
                                      "chat_id":chat_id,
                                      "completion_tokens":message_tokens_len,
                                    }
                                      ,schema=self.chat_ml_schema)
            # createa  list of ["chatml-" + str(uuid.uuid4())] with the same length as the number of messages
            
            message_df = nested_df.unnest("chat_ml")
            self.df = pl.concat([chat.df,message_df],how="diagonal",)
            return message_df                 
      
      def add_message_from_results_dict(self, results_dict: dict, prompt: List[Dict[str, str]]):
            
            data = {"role":[results_dict["choices"][0]["message"]["role"]],
                        "content":[results_dict["choices"][0]["message"]["content"]],
                        "author":[results_dict["model"]],
                        "message_id":[results_dict["id"]],
                        "chat_id":self.chat_id,
                        "prompt":[prompt],
                        "usage":[results_dict["usage"]],
                        "time_stamps":[datetime.fromtimestamp(results_dict["created"])],
            
                        }
            results_df = pl.DataFrame(data=data,schema=self.chat_nested_schema).unnest("usage")
            self.df = pl.concat([self.df,results_df],how="vertical")
            return results_df
      
      def fifo_filter(self, n: int, keep_first: bool = False):
            #extracts the last message sent by the user if the user is the last message and append it at the end of the returned df
            add_last = False
            if list(self.df[-1]["role"]) == ["user"]:
                  df_last_user_message = self.df.filter(pl.col("role") == "user")[-1]
                  #remove from df the last message sent by the user
                  df = self.df.filter(pl.col("message_id") != df_last_user_message["message_id"])
                  
                  n = n - df_last_user_message["completion_tokens"].sum()
                  add_last = True
            else:
                  df = self.df
                  
            #starts by removing messages that are sent by the system 
            df_no_system = df.filter(pl.col("role") != "system")
            df_system = df.filter(pl.col("role") == "system")

            
            if keep_first:
                  df_first = df_no_system.filter(pl.col("role") == "user")[0]
                  df_system = pl.concat([df_system,df_first],how="vertical")
                  df_no_system = df_no_system.filter(pl.col("message_id") != df_first["message_id"])
            system_tokens = df_system["completion_tokens"].sum()
            n = n - system_tokens
            
            cumulative_df = df_no_system.with_columns(pl.col("completion_tokens").reverse().cum_sum().reverse().alias("cumulative_completion_tokens_from_last_msg"))
            fifo_df = cumulative_df.filter(pl.col("cumulative_completion_tokens_from_last_msg") <= n).select(pl.all().exclude("cumulative_completion_tokens_from_last_msg"))
            if add_last:
                  fifo_df = pl.concat([fifo_df,df_last_user_message],how="vertical")

            return pl.concat([df_system,fifo_df],how="vertical")

      
      def to_chat_ml(self,df: Optional[pl.DataFrame]=None):
            if df is None:
                  df = self.df
            return df.select(pl.struct(["role","content"]).alias("chat_ml")).to_dict(as_series=False)["chat_ml"]
      
      def reply(self, message: str):
            user_as_chat_ml = {"role":"user","content":message}
            self.add_from_chat_ml([user_as_chat_ml],time_stamps=[datetime.now()],author="user")
            df_prompts = self.fifo_filter(self.ctx-self.output_reserved,keep_first=True)
            prompt = self.to_chat_ml(df_prompts)
            results = self.llm.create_chat_completion(
                  messages = prompt,
                  max_tokens = None,
                 
            )
            self.add_message_from_results_dict(results, prompt=prompt)
            return results

            
chat = ChatFrame(llm)
message_df = chat.add_from_chat_ml(messages)
results_df = chat.add_message_from_results_dict(results,prompt=messages)
new_user_message = {"role":"user", "content":"What is gang gang?"}
new_user_df = chat.add_from_chat_ml([new_user_message])


In [18]:
chat.reply("And how do I concatenate multiple columns into a single column row?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2577.84 ms
llama_print_timings:      sample time =      23.52 ms /   311 runs   (    0.08 ms per token, 13222.23 tokens per second)
llama_print_timings: prompt eval time =    2336.29 ms /    36 tokens (   64.90 ms per token,    15.41 tokens per second)
llama_print_timings:        eval time =   13691.45 ms /   310 runs   (   44.17 ms per token,    22.64 tokens per second)
llama_print_timings:       total time =   16430.61 ms


{'id': 'chatcmpl-dfaefba3-02b1-4402-b12c-c3521d864e05',
 'object': 'chat.completion',
 'created': 1704810095,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '"Gang Gang" is an internet slang term that has gained popularity in recent years, often used to express excitement or enthusiasm. It can be seen as a modern version of the phrase "Yahoo!" or "Woo-hoo!". The origin of the term is unclear, but it\'s believed to have started on social media platforms and online forums.\n\nTo concatenate multiple columns into a single column row in Python using pandas, you can use the `apply` function along with the `join` method. Here\'s an example:\n\n```python\nimport pandas as pd\n\n# Create a sample DataFrame\ndata = {\'Name\': [\'John\', \'Jane\'],\n        \'Age\': [25, 30],\n        \'City\': [\'New York\', \'San Francisco\']}\ndf = 

In [8]:
chat.reply("puoi tradurla in italiano?")

Llama.generate: prefix-match hit


shape: (1,)
Series: 'message_id' [str]
[
	"chatml-412af2e…
]



llama_print_timings:        load time =    2418.72 ms
llama_print_timings:      sample time =      10.04 ms /   131 runs   (    0.08 ms per token, 13054.31 tokens per second)
llama_print_timings: prompt eval time =     502.55 ms /    19 tokens (   26.45 ms per token,    37.81 tokens per second)
llama_print_timings:        eval time =    5798.74 ms /   130 runs   (   44.61 ms per token,    22.42 tokens per second)
llama_print_timings:       total time =    6467.75 ms


{'id': 'chatcmpl-80a39be1-1e4c-42eb-9c73-d0406ed56a07',
 'object': 'chat.completion',
 'created': 1704801153,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Polars è una libreria Python veloce, efficiente e facile da usare per il processing dei dati. Fornisce un oggetto DataFrame simile a pandas ma ottimizzato per la velocità e l'efficienza della memoria. Polars utilizza Rust come linguaggio di base, che gli permette di eseguire calcoli molto più velocemente rispetto ad altre librerie Python. La libreria supporta varie operazioni come filtro, groupby, join e altro su grandi dataset con facilità."},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 460,
  'completion_tokens': 130,
  'total_tokens': 590}}

In [9]:
chat.df

role,content,time_stamps,author,message_id,chat_id,prompt,prompt_tokens,completion_tokens,total_tokens
str,str,datetime[μs],str,str,str,list[struct[2]],i64,i64,i64
"""system""","""You are an ass…",2024-01-09 12:52:19.894352,"""import""","""chatml-b8e8ca7…","""99681ca2-f45c-…",null,null,16,null
"""user""","""What is polars…",2024-01-09 12:52:19.894354,"""import""","""chatml-412af2e…","""99681ca2-f45c-…",null,null,10,null
"""assistant""",""" Polars is a f…",2024-01-09 12:52:13,"""/Users/tommaso…","""chatcmpl-4107d…","""99681ca2-f45c-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}]",37,86,123
"""user""","""What is gang g…",2024-01-09 12:52:19.895671,"""import""","""chatml-9b81183…","""99681ca2-f45c-…",null,null,7,null
"""user""","""And how do I c…",2024-01-09 12:52:19.899078,"""user""","""chatml-1e988b9…","""99681ca2-f45c-…",null,null,17,null
"""assistant""",""" Gang Gang ref…",2024-01-09 12:52:19,"""/Users/tommaso…","""chatcmpl-9a069…","""99681ca2-f45c-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}, … {""user"",""And how do I concatenate multiple columns into a single column row?""}]",159,282,441
"""user""","""puoi tradurla …",2024-01-09 12:52:33.328149,"""user""","""chatml-14fd592…","""99681ca2-f45c-…",null,null,10,null
"""assistant""","""Polars è una l…",2024-01-09 12:52:33,"""/Users/tommaso…","""chatcmpl-80a39…","""99681ca2-f45c-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}, … {""user"",""puoi tradurla in italiano?""}]",460,130,590


In [10]:
pl.concat([chat.df,message_df],how="diagonal",)

role,content,time_stamps,author,message_id,chat_id,prompt,prompt_tokens,completion_tokens,total_tokens
str,str,datetime[μs],str,str,str,list[struct[2]],i64,i64,i64
"""system""","""You are an ass…",2024-01-09 12:52:19.894352,"""import""","""chatml-b8e8ca7…","""99681ca2-f45c-…",null,null,16,null
"""user""","""What is polars…",2024-01-09 12:52:19.894354,"""import""","""chatml-412af2e…","""99681ca2-f45c-…",null,null,10,null
"""assistant""",""" Polars is a f…",2024-01-09 12:52:13,"""/Users/tommaso…","""chatcmpl-4107d…","""99681ca2-f45c-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}]",37,86,123
"""user""","""What is gang g…",2024-01-09 12:52:19.895671,"""import""","""chatml-9b81183…","""99681ca2-f45c-…",null,null,7,null
"""user""","""And how do I c…",2024-01-09 12:52:19.899078,"""user""","""chatml-1e988b9…","""99681ca2-f45c-…",null,null,17,null
"""assistant""",""" Gang Gang ref…",2024-01-09 12:52:19,"""/Users/tommaso…","""chatcmpl-9a069…","""99681ca2-f45c-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}, … {""user"",""And how do I concatenate multiple columns into a single column row?""}]",159,282,441
"""user""","""puoi tradurla …",2024-01-09 12:52:33.328149,"""user""","""chatml-14fd592…","""99681ca2-f45c-…",null,null,10,null
"""assistant""","""Polars è una l…",2024-01-09 12:52:33,"""/Users/tommaso…","""chatcmpl-80a39…","""99681ca2-f45c-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}, … {""user"",""puoi tradurla in italiano?""}]",460,130,590
"""system""","""You are an ass…",2024-01-09 12:52:19.894352,"""import""","""chatml-b8e8ca7…","""99681ca2-f45c-…",null,null,16,null


In [11]:
chat.df.select(pl.struct([pl.col("role"),pl.col("content")]).alias("chat_ml")).to_dict(as_series=False)

{'chat_ml': [{'role': 'system',
   'content': 'You are an assistant who perfectly describes python packages. GANG GANG'},
  {'role': 'user', 'content': 'What is polars library in python?'},
  {'role': 'assistant',
   'content': ' Polars is a fast, efficient, and easy-to-use data processing library for Python. It provides a DataFrame object similar to pandas but optimized for speed and memory efficiency. Polars uses Rust as its core language, which allows it to perform computations much faster than other Python libraries. The library supports various operations like filtering, groupby, joins, and more on large datasets with ease.'},
  {'role': 'user', 'content': 'What is gang gang?'},
  {'role': 'user',
   'content': 'And how do I concatenate multiple columns into a single column row?'},
  {'role': 'assistant',
   'content': ' Gang Gang refers to the popular internet slang used by fans of the television show "The Office" as a catchphrase for camaraderie and teamwork. It has since been a

In [12]:
id = str(uuid.uuid4())
print(type(id))
pl.lit(id)

<class 'str'>


<Expr ['String(4c9003e8-0527-4f1c-81a8…'] at 0x11C9A5C10>

In [13]:
# df_single = pl.DataFrame(data={"chat_id": [1]})
df_single = df.with_columns(pl.lit(1).alias("chat_id"))
df_single

NameError: name 'df' is not defined

In [ ]:
from llama_cpp import Llama


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3724.28 ms
llama_print_timings:      sample time =       8.34 ms /   103 runs   (    0.08 ms per token, 12347.16 tokens per second)
llama_print_timings: prompt eval time =    1077.72 ms /    23 tokens (   46.86 ms per token,    21.34 tokens per second)
llama_print_timings:        eval time =   12324.44 ms /   102 runs   (  120.83 ms per token,     8.28 tokens per second)
llama_print_timings:       total time =   13537.53 ms


In [ ]:
results

{'id': 'chatcmpl-2e9e5958-e306-43f1-b92c-d304c817da98',
 'object': 'chat.completion',
 'created': 1704751139,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': " Polars is a fast, efficient, and easy-to-use data processing library for Python. It provides high-performance DataFrame operations, similar to pandas, but with optimized memory usage and faster execution times. Polars is built on top of the Rust programming language, which allows it to leverage the power of Rust's performance and safety features. The library includes support for reading and writing various file formats, handling missing data, groupby operations, joins, and more."},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 37, 'completion_tokens': 102, 'total_tokens': 139}}

In [ ]:
{'id': 'chatcmpl-eab5813e-3c8a-474b-bbee-636b9385d955',
 'object': 'chat.completion',
 'created': 1704751049,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': ' The Polars library in Python is a high-performance DataFrame library that provides fast and efficient data processing capabilities. It is designed to handle large datasets with ease, offering features such as lazy evaluation, parallel execution, and memory optimization. Polars supports various data types, including integers, floats, strings, booleans, and timestamps, making it suitable for a wide range of applications in data science, machine learning, and analytics.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 33, 'completion_tokens': 92, 'total_tokens': 125}}

In [ ]:
[results_dict["choices"][0]["message"]["content"]]

[" Polars is a fast, efficient, and easy-to-use data processing library for Python. It provides high-performance DataFrame operations, similar to pandas, but with optimized memory usage and faster execution times. Polars is built on top of the Rust programming language, which allows it to leverage the power of Rust's performance and safety features. The library includes support for reading and writing various file formats, handling missing data, groupby operations, joins, and more."]

In [ ]:
messages

[{'role': 'system',
  'content': 'You are an assistant who perfectly describes python packages.'},
 {'role': 'user', 'content': 'What is polars library in python?'}]

In [ ]:
messages_series = pl.Series(messages)
messages_series.


OrderedDict([('', Struct({'role': String, 'content': String}))])

In [ ]:
schema = {"prompt:": pl.List(pl.Struct({"role":pl.Utf8,"content":pl.Utf8})),
          "role":pl.Utf8,
            "content":pl.Utf8,
            "author":pl.Utf8,
            "message_id":pl.Utf8,}
data = {"prompt:": [messages_series],
        "role"}
df = pl.DataFrame(data=data,schema=schema)
df

prompt:
list[struct[2]]
"[{""system"",""You are an assistant who perfectly describes python packages.""}, {""user"",""What is polars library in python?""}]"


In [ ]:
chat_schema = {"role":pl.Utf8,
                   "content":pl.Utf8,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   "author":pl.Utf8,
                   "prompt": pl.List(pl.Struct({"role":role_dtype,"content":pl.Utf8})),
                   "usage": pl.Struct({'prompt_tokens': pl.Int64, 'completion_tokens': pl.Int64, 'total_tokens': pl.Int64})
}
def add_message_from_results_dict(self, results_dict: dict, prompt: List[Dict[str, str]]):
    data = {"role":[results_dict["choices"][0]["message"]["role"]],
            "content":[results_dict["choices"][0]["message"]["content"]],
            "author":[results_dict["model"]],
            "message_id":[results_dict["id"]],
            "chat_id":self.chat_id,
            "prompt":[prompt],
            "usage":[results_dict["usage"]]
    
            }
    df = pl.DataFrame(data=data,schema=chat_schema)
    self.df = pl.concat([self.df,df],how="vertical")


role,content,author,message_id,chat_id,prompt,usage
str,str,str,str,str,list[struct[2]],struct[3]
"""assistant""",""" Polars is a f…","""/Users/tommaso…","""chatcmpl-2e9e5…",null,"[{""system"",""You are an assistant who perfectly describes python packages.""}, {""user"",""What is polars library in python?""}]","{37,102,139}"
